# IOU Service Area Maps
The following notebook provides an example of how to subset for a specific area. 

In [ ]:
# imports
import pandas as pd
import geopandas as gpd
from figure_utils import get_service_area_map

Read in the station list and relevant shapefiles.

In [ ]:
# Read in merge station list
stn_list = pd.read_csv("s3://wecc-historical-wx/4_merge_wx/all_network_stationlist_merge.csv")

In [ ]:
# Convert it into geodataframe
stns_gdf = gpd.GeoDataFrame(
    stn_list,
    geometry=gpd.points_from_xy(stn_list.longitude, stn_list.latitude, crs="EPSG:4326"),
)

In [ ]:
# Read in IOU service territories shapefile -- Replace this file if you are interested in a different region!
service_territories = gpd.read_file(
    "s3://wecc-historical-wx/0_maps/California_Natural_Gas_Service_Area/"
)
service_territories = service_territories.to_crs(stns_gdf.crs)  # Convert to station CRS

Next, we'll identify which stations are within our desired area and visualize them. 

In [ ]:
# Identify which region we're interested in -- we'll demo with PG&E service territory
pge_geom = service_territories[service_territories["ABR"] == ("PG&E")]

In [ ]:
# Filter down to stations that are in the service territory
stns_gdf["intersects"] = stns_gdf.intersects(pge_geom.unary_union)

In [ ]:
# Get just those stations, drop the others
pge_stns = stns_gdf[stns_gdf["intersects"] == True].reset_index(
    drop=True
)

Let's look at a map!

In [ ]:
get_service_area_map('PG&E',save_local=False)